In [1]:
# import libraries 
import pandas as pd
import numpy as np
import math
from scipy.stats import kurtosis, skew

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.metrics import r2_score, mean_squared_error

from sklearn.preprocessing import OneHotEncoder, StandardScaler, MaxAbsScaler, MinMaxScaler
from sklearn.impute import SimpleImputer

from sklearn.pipeline import Pipeline

# this will allow us to visualize the pipeline (may not be available in learn-env)
from sklearn import set_config
set_config(display= 'diagram')

from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, \
RandomizedSearchCV, StratifiedKFold, KFold
from sklearn.feature_selection import SelectFromModel

from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, \
AdaBoostClassifier, GradientBoostingClassifier

from pickle import dump, load

# ignore warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_columns', None)

np.random_state = 42

In [2]:
# load batting_basic
batting_basic = pd.read_csv('Data/batting_basic', index_col = 0)
batting_basic = batting_basic.drop(columns = ['Season', 'Name'])
batting_basic.head()

,POS,Team,Age,Salary,G,AB,PA,AVG,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,OBP,SLG,OPS
0,OF,CHW,27.0,255000.0,80.0,215.0,242.0,0.274,59.0,40.0,15.0,1.0,3.0,31.0,29.0,21.0,1.0,38.0,2.0,1.0,2.0,2.0,2.0,1.0,0.343,0.395,0.738
1,2B,NYM,31.0,500000.0,79.0,157.0,173.0,0.217,34.0,20.0,7.0,1.0,6.0,22.0,12.0,14.0,2.0,51.0,1.0,1.0,0.0,2.0,1.0,1.0,0.283,0.389,0.672
5,OF,MIA,24.0,327000.0,111.0,255.0,281.0,0.212,54.0,35.0,12.0,2.0,5.0,39.0,24.0,18.0,2.0,78.0,3.0,1.0,4.0,2.0,6.0,5.0,0.271,0.333,0.604
6,OF,MIA,25.0,327000.0,35.0,76.0,80.0,0.197,15.0,10.0,3.0,0.0,2.0,16.0,5.0,2.0,0.0,22.0,2.0,0.0,0.0,1.0,7.0,1.0,0.238,0.316,0.553
7,OF,HOU,26.0,327000.0,34.0,55.0,60.0,0.309,17.0,10.0,5.0,0.0,2.0,10.0,5.0,1.0,0.0,23.0,2.0,1.0,1.0,0.0,5.0,2.0,0.339,0.509,0.848


## Train-Test Split

In [3]:
# feature selection, target variable = 'Salary'
X = batting_basic.drop(['Salary'], axis = 1)
y = batting_basic['Salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 42)

In [4]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8568, 26)
(2856, 26)
(8568,)
(2856,)


## Preprocessing

In [5]:
# load the transformer
bb_ct = load(open('pkl/bb_ct.pkl', 'rb'))

In [6]:
# inspect the preprocessed batting basic X train aka pp_bb_X_train
pp_bb_X_train = pd.DataFrame(bb_ct.fit_transform(X_train))
pp_bb_X_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.615324,-1.277737,-1.320185,-1.278757,-3.256567,-1.315740,-1.280789,-1.271027,-0.729205,-0.941820,-1.214658,-1.213309,-0.991055,-0.532858,-0.847974,-0.571759,-0.969914,2.033278,-1.163921,-0.602031,-0.721621,-2.163013,-2.929161,-2.762856
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.101455,-0.639005,-0.457531,-0.488064,-0.088735,-0.494367,-0.198899,-0.764249,-0.729205,-0.941820,-0.690033,-0.790427,-0.573329,-0.270018,-0.671409,-0.291008,-0.545728,-0.330878,-0.621142,0.788319,0.813442,-0.254618,-1.096442,-0.814301
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.372258,0.308081,0.008623,-0.026033,-0.607107,-0.257081,-0.115677,-0.173007,-0.729205,-0.639171,-0.296564,-0.367546,-0.406239,-0.532858,-0.368725,-0.571759,0.302643,1.357805,0.464415,-0.349240,0.429676,-0.820668,-0.907502,-0.913908
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.572267,-0.110399,-0.039600,-0.064138,0.410439,-0.019795,-0.087936,-0.341933,-0.729205,0.773187,-0.067040,0.836040,-0.322694,-0.270018,-0.620962,0.831998,0.302643,-0.668614,0.645342,-0.602031,-0.337856,0.262912,0.972451,0.735828
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.858389,1.233142,1.171330,1.236218,0.160852,0.947600,0.605583,1.178402,-0.252744,1.681132,1.178945,0.998686,1.389981,-0.270018,1.043799,1.955003,1.151015,-0.668614,1.188121,-0.475635,-0.721621,0.521678,0.783511,0.723377


In [7]:
pp_bb_X_train.shape

(8568, 61)

Now lets do the same transformation for the `X_test`

In [8]:
# inspect the preprocessed batting basic X train aka pp_bb_X_test
pp_bb_X_test = pd.DataFrame(bb_ct.transform(X_test))
pp_bb_X_test.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.615324,-1.475964,-1.288037,-1.269231,-1.394266,-1.224476,-1.225308,-1.186564,-0.729205,-0.740054,-1.050713,-0.953074,-0.907510,-0.532858,-1.074987,-0.010256,-0.969914,-0.668614,-0.982995,-0.475635,-0.721621,-0.060544,-0.605198,-0.422100
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.344521,-1.101535,-0.795092,-0.802436,-0.107934,-0.768158,-0.809196,-0.848712,-0.252744,-0.235640,-0.657244,-0.595251,-0.573329,-0.532858,-0.141712,-0.852510,-0.969914,-0.330878,-0.982995,0.029947,0.045910,-0.012026,0.528442,0.343627
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.329201,1.519470,1.841093,1.745882,1.831164,2.517337,2.935806,2.191958,2.129559,0.167891,1.605204,2.234801,0.805165,1.307020,0.438432,-0.291008,2.847758,-0.668614,1.911826,0.156342,0.045910,1.168591,0.859087,1.015972
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.615324,1.541495,1.514249,1.822093,-0.319123,0.984106,0.355916,1.516254,1.653098,1.983780,1.539625,1.551685,3.687472,1.044181,2.834678,2.235754,1.575201,-0.668614,0.645342,-0.349240,0.045910,0.909826,0.670147,0.798082
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.858389,0.219980,0.072920,0.116864,0.948010,0.235743,0.133990,0.502697,0.700177,0.067008,-0.034251,0.413158,0.220349,0.255661,-0.091265,1.112749,1.151015,-0.330878,-0.259290,-0.475635,-0.721621,1.006863,0.934663,1.003521


In [9]:
pp_bb_X_test.shape

(2856, 61)

**Log Transforming Salary**

In [10]:
log_y_train = np.log(y_train)
log_y_test = np.log(y_test)

## Modeling

In [11]:
# import my helper function to get model results
from functions import *

**1. Linear Regression**

In [12]:
# build pipeline 
steps = [('preprocess', bb_ct), 
         ('model', LinearRegression())]

lin_reg_pipeline = Pipeline(steps)

lin_reg_pipeline.fit(X_train, log_y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('nominalpipe',
                                                  Pipeline(steps=[('onehotenc',
                                                                   OneHotEncoder(sparse=False))]),
                                                  Index(['POS', 'Team'], dtype='object')),
                                                 ('numpipe',
                                                  Pipeline(steps=[('numnorm',
                                                                   StandardScaler())]),
                                                  Index(['Age', 'G', 'AB', 'PA', 'AVG', 'H', '1B', '2B', '3B', 'HR', 'R', 'RBI',
       'BB', 'IBB', 'SO', 'HBP', 'SF', 'SH', 'GDP', 'SB', 'CS', 'OBP', 'SLG',
       'OPS'],
      dtype='object'))])),
                ('model', LinearRegression())])

In [ ]:
# instantiate the model
lr_model = Pipeline(steps).steps[1][1]  # gets the class name of the model

# get model results
model_results(lr_model, pp_bb_X_train, log_y_train, pp_bb_X_test, log_y_test)

**2. Ridge Regression (Adding Regularization)**

In [ ]:
# build pipeline 
rr_steps = [('preprocess', bb_ct), 
            ('model', Ridge(random_state = 42))]

ridge_pipeline = Pipeline(rr_steps)

# define parameters in the grid
ridge_grid = {'model__alpha': [1e-3, 1e-2, 1e-1, 1, 10, 10**2],
              'model__solver': ['lbfgs', 'svd', 'saga']}  

# get best R2 score from grid search
ridge_model = GridSearchCV(estimator = ridge_pipeline,
                           param_grid = ridge_grid,
                           scoring = 'r2')

# fit and give me the best params
ridge_model.fit(X_train, log_y_train)
ridge_model.best_params_

In [ ]:
# instantiate the model
ridge_model = ridge_pipeline.steps[1][1]  # gets the class name of the model

# get model results
model_results(ridge_model, pp_bb_X_train, log_y_train, pp_bb_X_test, log_y_test)

**3. Support Vector Machine**

In [49]:
from sklearn import svm

In [51]:
# build pipeline 
svm_steps = [('preprocess', bb_ct), 
            ('model', svm.SVR())]

svm_pipeline = Pipeline(svm_steps)

# define parameters in the grid
svm_grid = {'model__C': [0.1, 1, 10],
            'model__tol': [1e-3, 1e-1, 1, 1e2]}  

# get best R2 score from grid search
svm_model = GridSearchCV(estimator = svm_pipeline,
                         param_grid = svm_grid,
                         scoring = 'r2')

# fit and give me the best params
svm_model.fit(X_train, log_y_train)
svm_model.best_params_

{'model__C': 1, 'model__tol': 0.1}

In [53]:
# instantiate the model
svm_model = svm_pipeline.steps[1][1]  # gets the class name of the model

# get model results
model_results(svm_model, pp_bb_X_train, log_y_train, pp_bb_X_test, log_y_test)

             SVR() RESULTS
SVR() Training R2:               0.6442
SVR() Test R2:                   0.5538
-------------------------------------------------------
SVR() Cross Validation R2:       0.5732
SVR() Cross Validation RMSE:    5594921
-------------------------------------------------------
SVR() Training RMSE:            3542893
SVR() Test RMSE:                3951044


**4. KNeighbors Regressor**

In [22]:
from sklearn.neighbors import KNeighborsRegressor

In [61]:
# build pipeline 
knr_steps = [('preprocess', bb_ct), 
            ('model', KNeighborsRegressor())]

knr_pipeline = Pipeline(knr_steps)

# define parameters in the grid
knr_grid = {'model__n_neighbors': [5, 6, 7],
            'model__weights': ['uniform', 'distance'],
            'model__algorithm': ['auto'],
            'model__p': [1, 2]}  

# get best R2 score from grid search
knr_model = GridSearchCV(estimator = knr_pipeline,
                           param_grid = knr_grid,
                           scoring = 'r2')

# fit and give me the best params
knr_model.fit(X_train, log_y_train)
knr_model.best_params_

{'model__algorithm': 'auto',
 'model__n_neighbors': 7,
 'model__p': 2,
 'model__weights': 'distance'}

In [62]:
# instantiate the model
knr_model = knr_pipeline.steps[1][1]  # gets the name of the model

# get model results
model_results(knr_model, pp_bb_X_train, log_y_train, pp_bb_X_test, log_y_test)

             KNeighborsRegressor() RESULTS
KNeighborsRegressor() Training R2:               0.63
KNeighborsRegressor() Test R2:                   0.411
-------------------------------------------------------
KNeighborsRegressor() Cross Validation R2:       0.4339
KNeighborsRegressor() Cross Validation RMSE:    3941424
-------------------------------------------------------
KNeighborsRegressor() Training RMSE:            3641273
KNeighborsRegressor() Test RMSE:                4401675


**5. Random Forest**

In [63]:
from sklearn.ensemble import RandomForestRegressor

In [64]:
# build pipeline 
rfr_steps = [('preprocess', bb_ct), 
            ('model', RandomForestRegressor(random_state = 42))]

rfr_pipeline = Pipeline(rfr_steps)

# define parameters in the grid
rfr_grid = {'model__n_estimators': [250, 280, 300],
            'model__min_samples_leaf': [1, 2, 3],
            'model__max_features': ['sqrt', 'log2']}  

# get best R2 score from grid search
rfr_model = GridSearchCV(estimator = rfr_pipeline,
                           param_grid = rfr_grid,
                           scoring = 'r2')

# fit and give me the best params
rfr_model.fit(X_train, log_y_train)
rfr_model.best_params_

{'model__max_features': 'sqrt',
 'model__min_samples_leaf': 1,
 'model__n_estimators': 280}

In [65]:
# instantiate the model
rfr_model = rfr_pipeline.steps[1][1]  # gets the name of the model

# get model results
model_results(rfr_model, pp_bb_X_train, log_y_train, pp_bb_X_test, log_y_test)

             RandomForestRegressor(random_state=42) RESULTS
RandomForestRegressor(random_state=42) Training R2:               0.9396
RandomForestRegressor(random_state=42) Test R2:                   0.5433
-------------------------------------------------------
RandomForestRegressor(random_state=42) Cross Validation R2:       0.5668
RandomForestRegressor(random_state=42) Cross Validation RMSE:    3628928
-------------------------------------------------------
RandomForestRegressor(random_state=42) Training RMSE:            2088458
RandomForestRegressor(random_state=42) Test RMSE:                4030428


**6. XGBoost**

In [68]:
import xgboost as xgb
from xgboost import XGBRegressor

In [84]:
# build pipeline 
xgb_steps = [('preprocess', bb_ct), 
            ('model', XGBRegressor())]

xgb_pipeline = Pipeline(xgb_steps)

# define parameters in the grid
xgb_grid = {'model__n_estimators': [180, 200],
            'model__max_depth': [3, 5, 8],
            'model__learning_rate': [0.01, 0.1]}  

# get best R2 score from grid search
xgb_model = GridSearchCV(estimator = xgb_pipeline,
                           param_grid = xgb_grid,
                           scoring = 'r2')

# fit and give me the best params
xgb_model.fit(X_train, log_y_train)
xgb_model.best_params_

{'model__learning_rate': 0.1,
 'model__max_depth': 5,
 'model__n_estimators': 200}

In [85]:
# instantiate the model
xgb_model = xgb_pipeline.steps[1][1]  # gets the name of the model

# get model results
model_results(xgb_model, pp_bb_X_train, log_y_train, pp_bb_X_test, log_y_test)

             XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...) RESULTS
XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
          

**MLP Regressor (Neural Network)**

In [87]:
from sklearn.neural_network import MLPRegressor

In [89]:
mlp = MLPRegressor()

mlp_model = mlp.fit(pp_bb_X_train, log_y_train)

model_results(mlp_model, pp_bb_X_train, log_y_train, pp_bb_X_test, log_y_test)

/Users/eric/miniforge3/envs/tf_miniforge/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/eric/miniforge3/envs/tf_miniforge/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


             MLPRegressor() RESULTS
MLPRegressor() Training R2:               0.7192
MLPRegressor() Test R2:                   0.5462
-------------------------------------------------------


/Users/eric/miniforge3/envs/tf_miniforge/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/eric/miniforge3/envs/tf_miniforge/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/eric/miniforge3/envs/tf_miniforge/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/eric/miniforge3/envs/tf_miniforge/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization has

MLPRegressor() Cross Validation R2:       0.5721
MLPRegressor() Cross Validation RMSE:    6003802
-------------------------------------------------------
MLPRegressor() Training RMSE:            3257245
MLPRegressor() Test RMSE:                4036693


/Users/eric/miniforge3/envs/tf_miniforge/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
